In [1]:
import tensorflow as tf
import numpy as np
import gc
import pandas as pd 
from tensorflow.keras.layers import ConvLSTM2D, Conv3D, Conv2D, Flatten, Dense, BatchNormalization
import tensorflow.math as M
from utils import *
from sklearn.model_selection import train_test_split
from video_loader import DeepFakeDualTransformer, DeepFakeTransformer
import sys
import pathlib
import datetime
import matplotlib.pyplot as plt

In [2]:
"""
(bsize, None, 16, 16, 88)
(bsize, None, 8, 8, 96)
(2, 1)
"""
class FeatureDifferentiatorModel(tf.keras.Model):
    def __init__(self):
        super(FeatureDifferentiatorModel, self).__init__()
        self.x1_8 = ConvLSTM2D(64, (3,3), strides=(1,1), 
                            padding='same', 
                            data_format='channels_last',
                            return_sequences=True,
                            dropout=0.2)
        self.x2_8 = ConvLSTM2D(128, (3,3), strides=(1,1), 
                                padding='same', 
                                data_format='channels_last',
                                return_sequences=False,
                                dropout=0.2)

        self.x1_16 = ConvLSTM2D(64, (3,3), strides=(1,1), 
                                    padding='same', 
                                    data_format='channels_last',
                                    return_sequences=True,
                                    dropout=0.2)
        self.x2_16 = ConvLSTM2D(128, (3,3), strides=(2,2), 
                            padding='same', 
                            data_format='channels_last',
                            return_sequences=False,
                            dropout=0.2)
        self.x_bn1 = tf.keras.layers.BatchNormalization()

        self.x_3 = tf.keras.layers.DepthwiseConv2D((3,3), 
                                              strides=(2,2), 
                                              depth_multiplier=2, 
                                              padding='same')
        self.x_bn2 = tf.keras.layers.BatchNormalization()
        self.x_4 = tf.keras.layers.DepthwiseConv2D((3,3), 
                                              strides=(2,2), 
                                              depth_multiplier=2, 
                                              padding='same')
        self.x_5 = tf.keras.layers.Conv2D(512, (1,1), strides=(2,2))
        self.x_bn3 = tf.keras.layers.BatchNormalization()
        self.x_7 = Dense(1, activation='sigmoid')

    def call(self, inputs):

        inputs16 = inputs[0]
        inputs8 = inputs[1]
        x8 = self.x1_8(inputs8)
        x8 = self.x2_8(x8)
        x16 = self.x1_16(inputs16)
        x16 = self.x2_16(x16)
        x_comb = tf.keras.layers.Add()([x16, x8])
        x = self.x_bn1(x_comb)
        x = self.x_3(x)
        x = self.x_bn2(x)
        x = self.x_4(x)
        x = self.x_5(x)
        x = self.x_bn3(x)
        x = tf.keras.layers.Flatten()(x)
        out = self.x_7(x)
        return out

In [3]:
model = FeatureDifferentiatorModel()
test_input8 = tf.keras.Input((None,8, 8, 96))
test_input16 = tf.keras.Input(( None, 16, 16, 88))
inputs = (test_input16, test_input8)
model(inputs)

<tf.Tensor 'feature_differentiator_model/Identity:0' shape=(None, 1) dtype=float32>

In [25]:
for wt in model.x_bn1.get_weights():
    print(wt.shape)

(128,)
(128,)
(128,)
(128,)


In [26]:
model.x_bn1.weights

[<tf.Variable 'feature_differentiator_model/batch_normalization/gamma:0' shape=(128,) dtype=float32, numpy=
 array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1.], dtype=float32)>,
 <tf.Variable 'feature_differentiator_model/batch_normalization/beta:0' shape=(128,) dtype=float32, numpy=
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.,

In [5]:
ckpt_reader = tf.train.load_checkpoint('models/202032811529_bzface_model_0.2_drop/')

In [6]:
d = ckpt_reader.get_variable_to_dtype_map()

In [21]:
l = []
for k in d:
    if 'OPTIMIZER' not in k and 'optimizer' not in k:
        print(k)
        l.append(k)

step/.ATTRIBUTES/VARIABLE_VALUE
net/x_bn3/moving_mean/.ATTRIBUTES/VARIABLE_VALUE
net/x2_8/cell/recurrent_kernel/.ATTRIBUTES/VARIABLE_VALUE
net/x_4/depthwise_kernel/.ATTRIBUTES/VARIABLE_VALUE
net/x_bn2/beta/.ATTRIBUTES/VARIABLE_VALUE
net/x_bn1/beta/.ATTRIBUTES/VARIABLE_VALUE
net/x1_8/cell/recurrent_kernel/.ATTRIBUTES/VARIABLE_VALUE
net/x1_16/cell/kernel/.ATTRIBUTES/VARIABLE_VALUE
net/x2_8/cell/kernel/.ATTRIBUTES/VARIABLE_VALUE
net/x_7/kernel/.ATTRIBUTES/VARIABLE_VALUE
net/x2_16/cell/bias/.ATTRIBUTES/VARIABLE_VALUE
net/x2_16/cell/recurrent_kernel/.ATTRIBUTES/VARIABLE_VALUE
net/x1_16/cell/bias/.ATTRIBUTES/VARIABLE_VALUE
net/x2_16/cell/kernel/.ATTRIBUTES/VARIABLE_VALUE
net/x1_8/cell/bias/.ATTRIBUTES/VARIABLE_VALUE
net/x_3/bias/.ATTRIBUTES/VARIABLE_VALUE
net/x_bn3/beta/.ATTRIBUTES/VARIABLE_VALUE
net/x_bn3/gamma/.ATTRIBUTES/VARIABLE_VALUE
net/x1_8/cell/kernel/.ATTRIBUTES/VARIABLE_VALUE
save_counter/.ATTRIBUTES/VARIABLE_VALUE
_CHECKPOINTABLE_OBJECT_GRAPH
net/x_5/kernel/.ATTRIBUTES/VARIABLE_VA

In [23]:
l.sort()
l

['_CHECKPOINTABLE_OBJECT_GRAPH',
 'net/x1_16/cell/bias/.ATTRIBUTES/VARIABLE_VALUE',
 'net/x1_16/cell/kernel/.ATTRIBUTES/VARIABLE_VALUE',
 'net/x1_16/cell/recurrent_kernel/.ATTRIBUTES/VARIABLE_VALUE',
 'net/x1_8/cell/bias/.ATTRIBUTES/VARIABLE_VALUE',
 'net/x1_8/cell/kernel/.ATTRIBUTES/VARIABLE_VALUE',
 'net/x1_8/cell/recurrent_kernel/.ATTRIBUTES/VARIABLE_VALUE',
 'net/x2_16/cell/bias/.ATTRIBUTES/VARIABLE_VALUE',
 'net/x2_16/cell/kernel/.ATTRIBUTES/VARIABLE_VALUE',
 'net/x2_16/cell/recurrent_kernel/.ATTRIBUTES/VARIABLE_VALUE',
 'net/x2_8/cell/bias/.ATTRIBUTES/VARIABLE_VALUE',
 'net/x2_8/cell/kernel/.ATTRIBUTES/VARIABLE_VALUE',
 'net/x2_8/cell/recurrent_kernel/.ATTRIBUTES/VARIABLE_VALUE',
 'net/x_3/bias/.ATTRIBUTES/VARIABLE_VALUE',
 'net/x_3/depthwise_kernel/.ATTRIBUTES/VARIABLE_VALUE',
 'net/x_4/bias/.ATTRIBUTES/VARIABLE_VALUE',
 'net/x_4/depthwise_kernel/.ATTRIBUTES/VARIABLE_VALUE',
 'net/x_5/bias/.ATTRIBUTES/VARIABLE_VALUE',
 'net/x_5/kernel/.ATTRIBUTES/VARIABLE_VALUE',
 'net/x_7/bias/

In [14]:
print(ckpt_reader.get_tensor('net/x1_16/cell/recurrent_kernel/.ATTRIBUTES/VARIABLE_VALUE').shape)
print(ckpt_reader.get_tensor('net/x1_16/cell/kernel/.ATTRIBUTES/VARIABLE_VALUE').shape)
print(ckpt_reader.get_tensor('net/x1_16/cell/bias/.ATTRIBUTES/VARIABLE_VALUE').shape)

(3, 3, 64, 256)
(3, 3, 88, 256)
(256,)


In [17]:
reccur = ckpt_reader.get_tensor('net/x1_16/cell/recurrent_kernel/.ATTRIBUTES/VARIABLE_VALUE')
kern = ckpt_reader.get_tensor('net/x1_16/cell/kernel/.ATTRIBUTES/VARIABLE_VALUE')
bias = ckpt_reader.get_tensor('net/x1_16/cell/bias/.ATTRIBUTES/VARIABLE_VALUE')

In [20]:
model.x1_16.set_weights(np.array([kern, reccur, bias]))

In [27]:
layer_list = ['x1_16', 'x1_8', 'x2_16', 'x2_8', 'x_3', 'x_4','x_5', 'x_7', 'x_bn1', 'x_bn2', 'x_bn3']

weights_dict = {}

for layer_pre in layer_list:
    if layer_pre in ['x1_16', 'x1_8', 'x2_16', 'x2_8']:
        recurr_name = f'net/{layer_pre}/cell/recurrent_kernel/.ATTRIBUTES/VARIABLE_VALUE'
        kern_name = f'net/{layer_pre}/cell/kernel/.ATTRIBUTES/VARIABLE_VALUE'
        bias_name = f'net/{layer_pre}/cell/bias/.ATTRIBUTES/VARIABLE_VALUE'
        r = ckpt_reader.get_tensor(recurr_name)
        k = ckpt_reader.get_tensor(kern_name)
        b = ckpt_reader.get_tensor(bias_name)
        weights_dict[layer_pre] = np.array([k, r, b])
    elif layer_pre in ['x_3', 'x_4']:
        kern_name = f'net/{layer_pre}/depthwise_kernel/.ATTRIBUTES/VARIABLE_VALUE'
        bias_name = f'net/{layer_pre}/bias/.ATTRIBUTES/VARIABLE_VALUE'
        k = ckpt_reader.get_tensor(kern_name)
        b = ckpt_reader.get_tensor(bias_name)
        weights_dict[layer_pre] = np.array([k, b])
    elif layer_pre in ['x_5', 'x_7']:
        kern_name = f'net/{layer_pre}/kernel/.ATTRIBUTES/VARIABLE_VALUE'
        bias_name = f'net/{layer_pre}/bias/.ATTRIBUTES/VARIABLE_VALUE'
        k = ckpt_reader.get_tensor(kern_name)
        b = ckpt_reader.get_tensor(bias_name)
        weights_dict[layer_pre] = np.array([k, b])
    elif layer_pre in ['x_bn1', 'x_bn2', 'x_bn3']:
        beta = f'net/{layer_pre}/beta/.ATTRIBUTES/VARIABLE_VALUE'
        gamma = f'net/{layer_pre}/gamma/.ATTRIBUTES/VARIABLE_VALUE'
        moving_mean = f'net/{layer_pre}/moving_mean/.ATTRIBUTES/VARIABLE_VALUE'
        moving_var = f'net/{layer_pre}/moving_variance/.ATTRIBUTES/VARIABLE_VALUE'
        b = ckpt_reader.get_tensor(beta)
        g = ckpt_reader.get_tensor(gamma)
        mm = ckpt_reader.get_tensor(moving_mean)
        mv = ckpt_reader.get_tensor(moving_var)
        weights_dict[layer_pre] = np.array([b, g, mm, mv])
            

In [29]:
model.x1_16.set_weights(weights_dict['x1_16'])
model.x2_16.set_weights(weights_dict['x2_16'])
model.x1_8.set_weights(weights_dict['x1_8'])
model.x2_8.set_weights(weights_dict['x2_8'])
model.x_3.set_weights(weights_dict['x_3'])
model.x_4.set_weights(weights_dict['x_4'])
model.x_5.set_weights(weights_dict['x_5'])
model.x_7.set_weights(weights_dict['x_7'])
model.x_bn1.set_weights(weights_dict['x_bn1'])
model.x_bn2.set_weights(weights_dict['x_bn2'])
model.x_bn3.set_weights(weights_dict['x_bn3'])